In [43]:
import pandas as pd
from hatchet import *
import os
import json
import utils
from state import State

In [44]:
dirname = "/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow"


In [45]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [46]:
def read_entire_gf(name):
        state = State()
        entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
        entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   

        with open(entire_graph_filepath, 'r') as entire_graphFile:
            entire_data = json.load(entire_graphFile)
            
        state.entire_gf = GraphFrame()
        from_literal(entire_data)

        state.entire_df = pd.read_csv(entire_df_filepath)
        state.entire_graph = state.entire_gf.graph

        # replace df['node'] from str to the Node object.
        state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)
        print(state.entire_df)

        return state

In [79]:
def get_metrics_from_df(name, nid):
    entire_df_filepath = dirname + '/' + dataset + '/entire_df.csv'
    entire_df = pd.read_csv(entire_df_filepath)
    
    node_df = entire_df[(entire_df.name == name) & (entire_df.nid == nid)]
    print(node_df.to_json())


In [84]:
def from_literal(graph_dict):
        """ Read graph from a dicts literal.
        """
        global lit_idx

        def parse_node_literal(child_dict, hparent, parent_callpath): 
            """ Create node_dict for one node and then call the function
                recursively on all children.
            """
            global lit_idx

            node_callpath = parent_callpath
            node_callpath.append(child_dict['name'])
            hnode = Node(lit_idx, tuple(node_callpath), hparent)
            metrics = get_metrics_from_df(graph_dict[i]['name'], graph_dict[i]['nid'])
            node_dicts.append(dict({'nid': lit_idx, 'node': hnode, 'name': child_dict['name'], 'metrics':metrics}))
            lit_idx += 1
            hparent.add_child(hnode)

            if 'children' in child_dict:
                for child in child_dict['children']:
                    parse_node_literal(child, hnode, list(node_callpath))

        list_roots = []
        node_dicts = []
        exc_metrics = []
        inc_metrics = []

        for i in range(len(graph_dict)):
            # start with creating a node_dict for the root
            root_callpath = []
            root_callpath.append(graph_dict[i]['name'])
            lit_idx = 0
            graph_root = Node(lit_idx, tuple(root_callpath), None)
            metrics = get_metrics_from_df(graph_dict[i]['name'], graph_dict[i]['nid'])
            node_dicts.append(dict({'nid': lit_idx, 'node': graph_root, 'name': graph_dict[i]['name'], 'metrics': metrics}))
            lit_idx += 1

            # call recursively on all children of root
            if 'children' in graph_dict[i]:
                for child in graph_dict[i]['children']:
                    parse_node_literal(child, graph_root, list(root_callpath))

            list_roots.append(graph_root)

            for key in graph_dict[i]['metrics'].keys():
                if '(inc)' in key:
                    inc_metrics.append(key)
                else:
                    exc_metrics.append(key)

        graph = Graph(list_roots)
        
#         dataframe = pd.DataFrame(data=node_dicts)
        print(node_dicts)
#         dataframe = pd.MultiIndex.from_frame(dataframe, names=['node', 'rank'])
#         dataframe.set_index(['node', 'rank'], drop=False, inplace=True)  
#         print(dataframe)

In [85]:
dataset = 'calc-pi'
state = read_entire_gf(dataset)
print(pd.__version__)

{"node":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"rank":{"0":0,"1":1,"2":2,"3":3},"time (inc)":{"0":999238.0,"1":999390.0,"2":1000306.0,"3":999308.0},"time":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},"nid":{"0":2,"1":2,"2":2,"3":2},"rank.1":{"0":0,"1":1,"2":2,"3":3},"file":{"0":"<unknown file>","1":"<unknown file>","2":"<unknown file>","3":"<unknown file>"},"line":{"0":0,"1":0,"2":0,"3":0},"module":{"0":"libmonitor.so.0.0.0","1":"libmonitor.so.0.0.0","2":"libmonitor.so.0.0.0","3":"libmonitor.so.0.0.0"},"name":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"node.1":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"type":{"0":"PF","1":"PF","2":"PF","3":"PF"},"n_index":{"0":0,"1":0,"2":0,"3":0},"mod_index":{"0":0,"1":0,"2":0,"3":0},"callees":{"0":"['main']","1":"['main']","2":"['main']","3":"['main']"},"callers":{"0":"[]","1":"[]","2":"[]","3":"[]"},"show_node":{"0":true,"1":tru

{"node":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"rank":{"0":0,"1":1,"2":2,"3":3},"time (inc)":{"0":999238.0,"1":999390.0,"2":1000306.0,"3":999308.0},"time":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},"nid":{"0":2,"1":2,"2":2,"3":2},"rank.1":{"0":0,"1":1,"2":2,"3":3},"file":{"0":"<unknown file>","1":"<unknown file>","2":"<unknown file>","3":"<unknown file>"},"line":{"0":0,"1":0,"2":0,"3":0},"module":{"0":"libmonitor.so.0.0.0","1":"libmonitor.so.0.0.0","2":"libmonitor.so.0.0.0","3":"libmonitor.so.0.0.0"},"name":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"node.1":{"0":"<program root>","1":"<program root>","2":"<program root>","3":"<program root>"},"type":{"0":"PF","1":"PF","2":"PF","3":"PF"},"n_index":{"0":0,"1":0,"2":0,"3":0},"mod_index":{"0":0,"1":0,"2":0,"3":0},"callees":{"0":"['main']","1":"['main']","2":"['main']","3":"['main']"},"callers":{"0":"[]","1":"[]","2":"[]","3":"[]"},"show_node":{"0":true,"1":tru

AttributeError: 'NoneType' object has no attribute 'roots'